In [1]:
import pandas as pd
import numpy as np

import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\anirban\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\anirban\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\anirban\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\anirban\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [2]:
#Load the data file and explore your data
osha = pd.read_csv("osha.txt",sep = "\t",header = None)

In [3]:
osha.head()

,0,1,2
0,201079928,Employee Is Burned By Forklift Radiator Fluid,At approximately 11:30 a.m. on November 13 2...
1,202561825,Employee Falls From Flatbed Trailer And Later...,On August 30 2013 Employee #1 was working f...
2,200361855,Two Workers Are Struck By Motor Vehicle And O...,On August 27 2013 Employees #1 and #2 of T...
3,200361863,Employee Is Struck By Bales Of Wire And Killed,On August 26 2013 Employee #1 with Lee Iro...
4,201079324,Employee Is Splashed With Hot Water And Is Bu...,On July 14 2013 Employee #1 vacuum pump tr...


In [4]:
osha.shape

(10000, 3)

In [5]:
osha.columns = ['id','title','Details']

In [6]:
osha.head()

,id,title,Details
0,201079928,Employee Is Burned By Forklift Radiator Fluid,At approximately 11:30 a.m. on November 13 2...
1,202561825,Employee Falls From Flatbed Trailer And Later...,On August 30 2013 Employee #1 was working f...
2,200361855,Two Workers Are Struck By Motor Vehicle And O...,On August 27 2013 Employees #1 and #2 of T...
3,200361863,Employee Is Struck By Bales Of Wire And Killed,On August 26 2013 Employee #1 with Lee Iro...
4,201079324,Employee Is Splashed With Hot Water And Is Bu...,On July 14 2013 Employee #1 vacuum pump tr...


### 1.	What are the major types of accidents reflected in the reports?

In [7]:
# Preprocessing

import nltk
from nltk import word_tokenize, pos_tag
from nltk.corpus import stopwords
mystopwords=stopwords.words("english") 
WNlemma = nltk.WordNetLemmatizer()

snowball = nltk.SnowballStemmer('english')

def pre_process(text):
    tokens = nltk.word_tokenize(text)
    tokens_pos = pos_tag(tokens)
    tokens =[]
    #Nouns
    for w, pos in tokens_pos:
        if pos[0] in ('N', 'V', 'J', 'R'):
            if pos in ('NN','NNPS','NNP','NNS', 'RBR', 'RBS', 'VBD', 'VBG', 'VBN', 'VBP', 'VBZ'):
                tokens.append(WNlemma.lemmatize(w.lower(), pos = pos[0].lower()))
            elif pos in ('JJR', 'JJS'):
                tokens.append(WNlemma.lemmatize(w.lower(), pos = 'a'))
            else: 
                tokens.append(w.lower())
    tokens=[WNlemma.lemmatize(t.lower()) for t in tokens]
    tokens=[ t for t in tokens if t not in mystopwords]
    tokens = [ t for t in tokens if len(t) >= 3 ]
    return(tokens)


text = osha.title
toks = text.apply(pre_process)

In [8]:
toks

0             [employee, burn, forklift, radiator, fluid]
1               [employee, fall, flatbed, trailer, later]
2                  [worker, strike, motor, vehicle, kill]
3                  [employee, struck, bale, wire, killed]
4                  [employee, splash, hot, water, burned]
                              ...                        
9995         [employee, kill, run, hot, metal, rail, car]
9996                      [employee, injure, flash, fire]
9997                             [employee, kill, struck]
9998    [employee, arm, amputated, caught, locomotive,...
9999                 [employee, kill, struck, locomotive]
Name: title, Length: 10000, dtype: object

In [9]:
import gensim 
from gensim import corpora
dictionary = corpora.Dictionary(toks)
print(dictionary)
dictionary.filter_extremes(no_below=2, no_above=0.9)

Dictionary(3305 unique tokens: ['burn', 'employee', 'fluid', 'forklift', 'radiator']...)


In [10]:
dtm = [dictionary.doc2bow(d) for d in toks]

lda = gensim.models.ldamodel.LdaModel(dtm, num_topics = 4, id2word = dictionary, passes=15,chunksize=32,random_state=10)

In [11]:
lda.show_topics(4)

[(0,
  '0.189*"employee" + 0.145*"kill" + 0.070*"burn" + 0.021*"arm" + 0.020*"loader" + 0.019*"crushed" + 0.019*"later" + 0.014*"hot" + 0.013*"strike" + 0.011*"trailer"'),
 (1,
  '0.119*"employee" + 0.055*"shock" + 0.055*"die" + 0.054*"electric" + 0.040*"crush" + 0.036*"injured" + 0.036*"leg" + 0.033*"hand" + 0.021*"fire" + 0.020*"locomotive"'),
 (2,
  '0.152*"employee" + 0.086*"fall" + 0.058*"finger" + 0.047*"fracture" + 0.038*"worker" + 0.025*"caught" + 0.024*"machine" + 0.022*"killed" + 0.020*"amputates" + 0.018*"ladder"'),
 (3,
  '0.188*"employee" + 0.121*"struck" + 0.108*"injure" + 0.054*"fall" + 0.038*"truck" + 0.018*"pin" + 0.015*"explosion" + 0.013*"metal" + 0.012*"injures" + 0.011*"head"')]

In [12]:
from gensim.models.coherencemodel import CoherenceModel
cm_umass = CoherenceModel(lda,  dictionary=dictionary, corpus=dtm, coherence='u_mass')
cm_cv = CoherenceModel(lda,  dictionary=dictionary, texts=toks, coherence='c_v')
lda_umass = cm_umass.get_coherence()
lda_cv = cm_cv.get_coherence()
print(lda_umass)
print(lda_cv)

-12.050795189101855
0.4627572162100224


The output of the LDA model indicates the major types of accidents.

In [13]:
from pyLDAvis import gensim
import pickle 
import pyLDAvis
#Visualize the topics
pyLDAvis.enable_notebook()
LDAvis_prepared = pyLDAvis.gensim.prepare(lda, dtm, dictionary)
pyLDAvis.display(LDAvis_prepared)

### Which Type of Accidents Has Largest Number of Occurences

In [14]:
# Get the topic distribution of documents
doc_topics = lda.get_document_topics(dtm)

from operator import itemgetter
#show the topic distributions for the first 5 docs, 
for i in range(0, 5):
    print(doc_topics[i])
    print(max(doc_topics[i], key=itemgetter(1))[0]) 

 
#Select the best topic (with highest score) for each document
top_topic = [max(t, key=itemgetter(1))[0] for t in doc_topics ]


C:\anaconda\envs\anirban\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


[(0, 0.6764053), (1, 0.04206894), (2, 0.23922543), (3, 0.042300336)]
0
[(0, 0.46446595), (1, 0.042108573), (2, 0.044584565), (3, 0.44884092)]
0
[(0, 0.7083278), (1, 0.041669257), (2, 0.20833439), (3, 0.04166855)]
0
[(0, 0.042360283), (1, 0.23445348), (2, 0.24381413), (3, 0.47937208)]
3
[(0, 0.47988287), (1, 0.23472343), (2, 0.24303867), (3, 0.042355023)]
0


In [15]:
import collections

dict(collections.Counter(top_topic))

C:\anaconda\envs\anirban\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


{0: 2774, 3: 2207, 1: 1743, 2: 3276}

In [16]:
import nltk
from nltk import word_tokenize, pos_tag
 
x = osha.title
type(x[0])
 
 
for sent in x:
    sent_pos = pos_tag(word_tokenize(sent))

C:\anaconda\envs\anirban\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [17]:
nltk.download('universal_tagset')

for s in x :    
    sent_pos2 = pos_tag(word_tokenize(s), tagset='universal')

C:\anaconda\envs\anirban\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
[nltk_data] Downloading package universal_tagset to
[nltk_data]     C:\Users\anirban\AppData\Roaming\nltk_data...
[nltk_data]   Package universal_tagset is already up-to-date!


In [18]:
wnl = nltk.WordNetLemmatizer()
wnl.lemmatize('born', pos = 'v')
def lemmaNVAR(wpos):
    lemmas = []
    for w, pos in wpos:
        if pos[0] in ('N', 'V', 'J', 'R'):
            if pos in ('NNS', 'RBR', 'RBS', 'VBD', 'VBG', 'VBN', 'VBP', 'VBZ'):
                lemmas.append(wnl.lemmatize(w.lower(), pos = pos[0].lower()))
            elif pos in ('JJR', 'JJS'):
                lemmas.append(wnl.lemmatize(w.lower(), pos = 'a'))
            else:
                lemmas.append(w.lower())
    return lemmas

C:\anaconda\envs\anirban\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [19]:
from nltk import pos_tag

for sent in x:
    sent_pos = pos_tag(word_tokenize(sent))
    print(lemmaNVAR(sent_pos))

C:\anaconda\envs\anirban\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


['employee', 'is', 'burn', 'forklift', 'radiator', 'fluid']
['employee', 'falls', 'from', 'flatbed', 'trailer', 'later', 'dies']
['worker', 'be', 'strike', 'motor', 'vehicle', 'be', 'kill']
['employee', 'is', 'struck', 'bales', 'wire', 'killed']
['employee', 'be', 'splash', 'hot', 'water', 'is', 'burned']
['employee', 'suffers', 'burns', 'move', 'soup']
['employee', 'injure', 'self', 'knife']
['foreman', 'be', 'fatally', 'crush', 'forklift', 'tips']
['employee', 'fracture', 'abdomen', 'run', 'tug', 'cart']
['employee', 'suffers', 'abdominal', 'fracture', 'fall', 'from', 'ladder']
['carpenter', 'injured', 'abdomen', 'saw', 'kicks', 'back']
['employee', 'abdomen', 'head', 'injury', 'slip', 'fall', 'tarp']
['employee', 'sustains', 'rupture', 'spleen', 'board', 'kicks', 'back']
['employee', 'body', 'be', 'caught', 'asphalt', 'machine', 'dies']
['logger', 'be', 'injure', 'butt', 'tree', 'kicks', 'out', 'strikes', 'him']
['worker', 'be', 'injure', 'struck', 'board', 'thrown', 'onto', 'steps'

['employee', 'be', 'severly', 'burn', 'explosion']
['employee', 'is', 'burned', 'use', 'acetylene', 'torch']
['worker', 'use', 'cutting', 'torch', 'is', 'killed', 'tank', 'explodes']
['employee', 'be', 'injure', 'fall', 'from', 'scissor', 'lift']
['employee', 'face', 'be', 'burn', 'motor', 'oil', 'ignites']
['employee', 'die', 'fall', 'through', 'floor', 'opening']
['employee', 'be', 'injure', 'struck', 'metal', 'door']
['employee', 'receives', 'burns', 'using', 'cutting', 'torch']
['employee', 'contacts', 'torch', 'flame', 'is', 'burned', 'later', 'dies']
['employee', 'be', 'kill', 'injure', 'explosion']
['employee', 'receive', 'degree', 'burns', 'from', 'flash', 'fire']
['employee', 'is', 'burned', 'cut', 'top', 'off', 'drum']
['employee', 'be', 'kill', 'fire', 'water', 'tank', 'truck']
['employee', 'is', 'burn', 'acetylene', 'explosion']
['employee', 'be', 'crush', 'asphyxiate', 'fall', 'rail', 'car', 'wall']
['employee', 'burn', 'natural', 'gas', 'line', 'ignites', 'explodes']
['em

['employee', 'be', 'pin', 'drug', 'truck']
['employee', 'amputates', 'finger', 'moving', 'valve']
['employee', 'amputates', 'fingertip', 'pneumatic', 'pruner']
['worker', 'amputate', 'thumb', 'air']
['employee', 'lacerates', 'arm', 'face', 'tire', 'expodes']
['employee', 'be', 'injure', 'tire', 'explodes']
['employee', 'dy', 'exposure', 'chlorine']
['employee', 'die', 'from', 'exhaust', 'flow', 'into', 'breathing', 'apparatus']
['employee', 'suffer', 'a', 'fractured', 'pelvis', 'crane', 'accident']
['employee', 'be', 'kill', 'caught', 'debarking', 'machine']
['carpenter', 'falls', 'from', 'bridge', 'pier', 'drowns']
['employee', 'dy', 'carbon', 'monoxide', 'exposure']
['diver', 'be', 'kill', 'pull', 'into', 'boat', 'propeller']
['employee', 'die', 'asphyxiation']
['employee', 'die', 'asphyxia', 'undersea', 'oil', 'rig']
['employee', 'injure', 'nitrogen', 'inhalation']
['employee', 'die', 'asphyxiation']
['employee', 'die', 'from', 'asphyxiation', 'sandblast', 'steel']
['employee', 'suf

['worker', 'be', 'burn', 'spilled', 'alcohol', 'ignites']
['employee', 'be', 'kill', 'fire']
['employee', 'falls', 'building', 'later', 'dies']
['worker', 'cutting', 'tops', 'off', 'metal', 'drumjs', 'die', 'explosion']
['employee', 'separating', 'panes', 'glass', 'is', 'burn', 'alcohol']
['employee', 'sustains', 'burns', 'isopropyl', 'alcohol', 'ignites']
['employee', 'be', 'find', 'dead', 'porta', 'potty']
['employee', 'is', 'burned', 'spray', 'paint']
['employee', 'killed', 'fall', 'from', 'communications', 'tower']
['employee', 'suffers', 'dermatitis', 'response', 'exposure']
['firefighter', 'experiences', 'allergic', 'reaction', 'bite']
['employee', 'die', 'from', 'bee', 'sting']
['employee', 'die', 'from', 'bee', 'stings']
['employee', 'be', 'injure', 'bitten', 'rattlesnake']
['employee', 'die', 'from', 'carbon', 'monoxide', 'exposure']
['employee', 'be', 'stung', 'wasp', 'later', 'dies']
['employee', 'die', 'from', 'asthma', 'attack']
['employee', 'die', 'from', 'allergic', 'rea

...

['employee', 'injure', 'burn', 'body', 'presser']
['employee', 'injure', 'crush', 'ironer', 'roller']
['employee', 'hand', 'injure', 'slip', 'foot', 'pedal']
['employee', 'arm', 'struck', 'head', 'garment', 'pressing', 'machine']
['employee', 'hand', 'crushed', 'iron', 'rollers']
['employee', 'hand', 'severely', 'burn', 'pant', 'press']
['employee', 'burn', 'iron', 'pressers']
['employee', 'fingertip', 'burn', 'steam', 'press']
['employee', 'hand', 'burned', 'ironing', 'machine']
['employee', 'fractures', 'leg', 'struck', 'pipe']
['farm', 'worker', 'fractures', 'leg', 'instal', 'irrigation', 'pipe']
['employee', 'injures', 'knee', 'instal', 'irrigation', 'pipe']
['employee', 'dy', 'heart', 'attack']
['employee', 'die', 'from', 'cardiac', 'arrest']
['worker', 'die', 'be', 'run', 'tractor', 'backhoe']
['employee', 'thumb', 'be', 'crush', 'silo', 'auger']
['crop', 'irrigation', 'worker', 'be', 'find', 'dead']
['employee', 'be', 'kill', 'impale', 'backhoe', 'teeth']
['employee', 'fracture'

['worker', 'be', 'hospitalize', 'heat', 'exhaustion', 'dehydration']
['worker', 'die', 'sustain', 'heat', 'stroke']
['employee', 'be', 'injure', 'pin', 'truck']
['employee', 'suffers', 'kidney', 'failure', 'training']
['employee', 'be', 'injure', 'fell', 'tree']
['employee', 'is', 'struck', 'fall', 'object', 'receives', 'injuries']
['employee', 'be', 'injure', 'excessive', 'training', 'exercise']
['employee', 'suffers', 'heat', 'stroke', 'renal', 'failure']
['employee', 'falls', 'from', 'roof', 'ruptures', 'kidney']
['employee', 'be', 'injure', 'fall', 'from', 'scaffold']
['employee', 'be', 'injure', 'arsine', 'gas']
['employee', 'is', 'burn', 'hot', 'kiln', 'feed']
['employee', 'die', 'from', 'possible', 'carbon', 'monoxide', 'poisoning']
['kiln', 'operator', 'dies', 'gas', 'explosion']
['employee', 'be', 'injure', 'caught', 'kiln']
['employee', 'be', 'injure', 'trip', 'fall']
['worker', 'die', 'confined', 'space', 'accident']
['employee', 'suffers', 'second']
['employee', 'be', 'kill

['employee', 'fractures', 'thumb', 'finger', 'lawn', 'mower', 'blade']
['employee', 'die', 'equipment', 'roll']
['employee', 'die', 'work', 'out', 'side', 'all', 'day']
['employee', 'lose', 'finger', 'mowing', 'accident']
['employee', 'amputate', 'finger', 'mower']
['employee', 'be', 'kill', 'crush', 'manlift']
['smelt', 'kettle', 'cover', 'smashes', 'worker', 'finger']
['employee', 'falls', 'from', 'elevation']
['employee', 'falss', 'feet', 'floor', 'whole']
['employee', 'injure', 'fingers', 'use', 'dryer', 'machine']
['electrican', 'burns', 'neck', 'troubleshoot', 'a', 'cable']
['worker', 'gets', 'shocked', 'remove', 'leads', 'from', 'transformer']
['employee', 'be', 'injure', 'crane', 'falls']
['employee', 'hospitalized', 'lead', 'poisoning']
['employee', 'became', 'sick', 'expose', 'fumes', 'lead']
['employee', 'died', 'sepsis']
['employee', 'struck', 'horizontal', 'beam']
['employee', 'engage', 'lead', 'paint', 'removal']
['frame', 'worker', 'falls', 'building', 'sustains', 'fract

['employee', 'struck', 'shoulder', 'fall', 'tree', 'branch']
['employee', 'kill', 'struck', 'limb', 'gate']
['employee', 'drowns', 'pond']
['employee', 'be', 'overcome', 'lime', 'fumes']
['employee', 'be', 'kill', 'remove', 'limestone', 'from', 'rail', 'car']
['employee', 'arm', 'is', 'amputated', 'caught', 'conveyor', 'belt']
['employee', 'is', 'asphyxiated', 'aspirating', 'lime', 'slurry']
['employee', 'die', 'due', 'severe', 'lime', 'burns']
['employee', 'kill', 'struck', 'transport', 'truck']
['employee', 'fatally', 'injured', 'pin', 'down', 'forklift']
['employee', 'arm', 'amputated', 'screw', 'conveyor']
['employee', 'burn', 'hazardous', 'material']
['employee', 'die', 'engulfed', 'limestone', 'bin']
['employee', 'suffers', 'burns', 'wet', 'lime', 'explodes', 'furnace']
['employee', 'burn', 'a', 'heated', 'lime', 'mixture']
['employee', 'suffers', 'chemical', 'burns', 'eyes', 'face']
['eye', 'burn', 'hydrated', 'lime']
['employee', 'chemically', 'burn', 'lime']
['conveyor', 'shut

['employee', 'foot', 'crushed', 'trailer']
['employee', 'killed', 'be', 'crush', 'heavy', 'machinery']
['employee', 'finger', 'severed', 'crushed', 'work']
['employee', 'partially', 'amputates', 'fingers', 'packing', 'machine']
['worker', 'amputate', 'finger', 'pallet', 'making', 'machine']
['employee', 'amputates', 'fingertip', 'machine']
['truck', 'driver', 'crossing', 'railroad', 'tracks', 'is', 'struck', 'train']
['employee', 'be', 'kill', 'crush', 'locomotive']
['employee', 'be', 'kill', 'struck', 'train']
['employee', 'leg', 'is', 'pin', 'wheel', 'assembly', 'later', 'dies']
['employee', 'finger', 'is', 'amputated', 'struck', 'fall', 'object']
['employee', 'be', 'kill', 'caught', 'between', 'train', 'car', 'door']
['logger', 'be', 'kill', 'collision', 'locomotive']
['employee', 'die', 'run', 'locomotive', 'fuel', 'tank']
['employee', 'be', 'kill', 'crushed', 'between', 'railroad', 'cars']
['employee', 'be', 'kill', 'run', 'hot', 'metal', 'rail', 'car']
['employee', 'be', 'injure'

In [20]:
from nltk.corpus import wordnet as wn
wn.synsets('employee')
print (wn.synsets('employee'))

[Synset('employee.n.01')]


C:\anaconda\envs\anirban\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [21]:
#print (wn.synsets('employee', pos = wn.NOUN))
ss = wn.synsets('employee', pos = wn.NOUN)[0]
print(ss.definition())
print(ss.examples())
print(ss.lemmas())
lem = ss.lemmas()[0]
print(lem.name())
print(ss.lemma_names())

a worker who is hired to perform a job
[]
[Lemma('employee.n.01.employee')]
employee
['employee']


C:\anaconda\envs\anirban\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [22]:
ss.hyponyms()

C:\anaconda\envs\anirban\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


[Synset('bartender.n.01'),
 Synset('clerk.n.01'),
 Synset('company_man.n.01'),
 Synset('copyist.n.01'),
 Synset('copywriter.n.01'),
 Synset('crewman.n.03'),
 Synset('deliveryman.n.01'),
 Synset('dining-room_attendant.n.01'),
 Synset('dispatcher.n.02'),
 Synset('dog_catcher.n.01'),
 Synset('floater.n.04'),
 Synset('floorwalker.n.01'),
 Synset('gardener.n.02'),
 Synset('gasman.n.01'),
 Synset('gofer.n.01'),
 Synset('hire.n.01'),
 Synset('hired_help.n.01'),
 Synset('hireling.n.01'),
 Synset('jobholder.n.01'),
 Synset('line_worker.n.01'),
 Synset('liveryman.n.01'),
 Synset('office_boy.n.01'),
 Synset('organization_man.n.01'),
 Synset('porter.n.06'),
 Synset('potboy.n.01'),
 Synset('public_servant.n.01'),
 Synset('registrar.n.01'),
 Synset('salesperson.n.01'),
 Synset('sandwichman.n.01'),
 Synset('spotter.n.01'),
 Synset('spotter.n.02'),
 Synset('staff_member.n.01'),
 Synset('stagehand.n.01'),
 Synset('stocktaker.n.01'),
 Synset('sweeper.n.01'),
 Synset('tollkeeper.n.01'),
 Synset('trainman

In [23]:
ss.hyponyms()[0].lemma_names()

C:\anaconda\envs\anirban\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


['bartender', 'barman', 'barkeep', 'barkeeper', 'mixologist']

In [24]:
hyps = list(set([w for s in ss.closure(lambda s:s.hyponyms()) for w in s.lemma_names()]))
occup_freq = []
for i in range(len(toks)):
    occup = []
    for word in toks[i]:
        if word in hyps:
            occup.append(word)
        occup_freq.extend(list(set(occup)))

C:\anaconda\envs\anirban\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [25]:
from nltk import FreqDist

fd_grain = FreqDist(occup_freq)
fd_grain.most_common(30)

C:\anaconda\envs\anirban\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


[('hand', 1195),
 ('loader', 107),
 ('jack', 93),
 ('laborer', 76),
 ('logger', 64),
 ('excavator', 59),
 ('harvester', 25),
 ('picker', 15),
 ('groundskeeper', 14),
 ('packer', 14),
 ('brakeman', 11),
 ('sweeper', 10),
 ('warehouseman', 9),
 ('cleaner', 8),
 ('clerk', 8),
 ('sweep', 8),
 ('wrangler', 5),
 ('stacker', 5),
 ('woman', 5),
 ('waitress', 5),
 ('blaster', 4),
 ('dockworker', 4),
 ('splitter', 4),
 ('sprayer', 2),
 ('sorter', 2),
 ('carrier', 1),
 ('mover', 1),
 ('feller', 1)]

In [26]:

#print (wn.synsets('employee', pos = wn.NOUN))
ss = wn.synsets('body_part', pos = wn.NOUN)[0]
print(ss.definition())
print(ss.examples())
print(ss.lemmas())
lem = ss.lemmas()[0]
print(lem.name())
print(ss.lemma_names())



any part of an organism such as an organ or extremity
[]
[Lemma('body_part.n.01.body_part')]
body_part
['body_part']


C:\anaconda\envs\anirban\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [27]:
ss.hyponyms()

C:\anaconda\envs\anirban\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


[Synset('abdomen.n.01'),
 Synset('adnexa.n.01'),
 Synset('ambulacrum.n.01'),
 Synset('ampulla.n.01'),
 Synset('apparatus.n.02'),
 Synset('area.n.03'),
 Synset('back.n.01'),
 Synset('buttock.n.01'),
 Synset('buttocks.n.01'),
 Synset('cannon.n.05'),
 Synset('dilator.n.01'),
 Synset('dock.n.06'),
 Synset('dorsum.n.02'),
 Synset('energid.n.01'),
 Synset('external_body_part.n.01'),
 Synset('feature.n.02'),
 Synset('flank.n.04'),
 Synset('fornix.n.01'),
 Synset('gaskin.n.01'),
 Synset('groove.n.03'),
 Synset('haunch.n.01'),
 Synset('hindquarters.n.02'),
 Synset('hip.n.01'),
 Synset('horseback.n.01'),
 Synset('joint.n.01'),
 Synset('lobe.n.01'),
 Synset('loin.n.02'),
 Synset('loins.n.02'),
 Synset('mentum.n.03'),
 Synset('organ.n.01'),
 Synset('partition.n.03'),
 Synset('process.n.05'),
 Synset('rectum.n.01'),
 Synset('rudiment.n.02'),
 Synset('saddle.n.06'),
 Synset('shank.n.02'),
 Synset('shin.n.01'),
 Synset('shoulder.n.01'),
 Synset('small.n.01'),
 Synset('structure.n.04'),
 Synset('stump

In [28]:
ss.hyponyms()[0].lemma_names()

C:\anaconda\envs\anirban\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


['abdomen', 'venter', 'stomach', 'belly']

In [29]:
hyps = list(set([w for s in ss.closure(lambda s:s.hyponyms()) for w in s.lemma_names()]))
hyps



C:\anaconda\envs\anirban\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


['cerebellar_hemisphere',
 'subthalamus',
 'cheek_muscle',
 'hindlimb',
 'left_coronary_artery',
 'fluke',
 'intercostal',
 'glandulae_sebaceae',
 'cardiac_muscle',
 'central_artery_of_the_retina',
 'cowl_muscle',
 'speech_organ',
 'coxcomb',
 'fascia',
 'os_temporale',
 'hypoglossal',
 'plexus_choroideus',
 'tarsal',
 'gob',
 'rhomboid_muscle',
 'sulcus_lateralis_cerebri',
 'male_genital_organ',
 'sigmoid_flexure',
 "Broca's_center",
 'vena_diploica',
 'vital_organ',
 'auditory_tube',
 'bronchial_tube',
 'arteria_lumbalis',
 'pussy',
 'nerve_plexus',
 'visual_system',
 'alveolar_bed',
 'family_jewels',
 'cortical_area',
 'alveolar_process',
 'arteria_communicans',
 'teres',
 'vena_supraorbitalis',
 'enarthrodial_joint',
 'musculus_deltoideus',
 'cell_organelle',
 'elbow_joint',
 'vena_cava',
 'cavity',
 'bile_duct',
 'pituitary_gland',
 'vena_angularis',
 'lesser_omentum',
 'pleopod',
 'pyramidal_motor_system',
 'systema_skeletale',
 'inferior_colliculus',
 'endothelium',
 'hypoblast'

In [33]:
# text = osha.desc
# toks = text.apply(pre_process)
injury_freq = []
for i in range(len(toks)):
    injury = []
    for word in toks[i]:
        if word in hyps:
            injury.append(word)
            injury_freq.extend(list(set(injury)))
print(injury_freq)

C:\anaconda\envs\anirban\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


['abdomen', 'abdominal', 'abdomen', 'abdomen', 'back', 'abdomen', 'head', 'abdomen', 'spleen', 'spleen', 'back', 'body', 'butt', 'abdomen', 'nail', 'abdomen', 'abdomen', 'ankle', 'leg', 'facial', 'arm', 'knee', 'rib', 'finger', 'lung', 'knee', 'finger', 'hand', 'hand', 'finger', 'leg', 'finger', 'heart', 'finger', 'structure', 'hand', 'hand', 'system', 'pot', 'thigh', 'hood', 'foot', 'thumb', 'thumb', 'grinder', 'wrist', 'hand', 'leg', 'hand', 'hand', 'arm', 'arm', 'hand', 'arm', 'arm', 'shaft', 'hand', 'hand', 'grinder', 'arm', 'hand', 'face', 'arm', 'arm', 'grinder', 'thumb', 'eye', 'tool', 'eye', 'finger', 'finger', 'finger', 'grinder', 'face', 'grinder', 'finger', 'hand', 'arm', 'arm', 'grinder', 'finger', 'pin', 'finger', 'neck', 'back', 'leg', 'foot', 'leg', 'leg', 'pin', 'pin', 'foot', 'pin', 'foot', 'valve', 'lid', 'hand', 'hand', 'arm', 'finger', 'eye', 'nail', 'arm', 'forearm', 'face', 'opening', 'leg', 'back', 'pelvis', 'back', 'leg', 'wrist', 'limb', 'head', 'face', 'back',

In [34]:
fd_grain = FreqDist(injury_freq)
fd_grain.most_common(30)

C:\anaconda\envs\anirban\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


[('finger', 942),
 ('hand', 380),
 ('leg', 357),
 ('arm', 300),
 ('head', 193),
 ('foot', 191),
 ('pin', 167),
 ('back', 146),
 ('thumb', 118),
 ('ankle', 91),
 ('heart', 89),
 ('face', 84),
 ('shaft', 70),
 ('eye', 66),
 ('hip', 65),
 ('wrist', 61),
 ('rib', 51),
 ('blade', 51),
 ('opening', 47),
 ('dock', 46),
 ('knee', 41),
 ('neck', 41),
 ('bridge', 41),
 ('system', 39),
 ('grinder', 39),
 ('left', 39),
 ('right', 38),
 ('valve', 35),
 ('toe', 35),
 ('pelvis', 33)]